# Old Data Augmentation

In [1]:
# import albumentations as A
# import cv2
# import os
# from pathlib import Path
# import pandas as pd

In [6]:
# labels_df = pd.read_csv('../data/fundus/MuReD/train_data.csv')
# labels_df['ID'] = labels_df['ID'].astype(str)
# class_counts = labels_df.iloc[:, 1:].sum(axis=0)
# class_counts

DR        396
NORMAL    395
MH        135
ODC       211
TSLN      125
ARMD      126
DN        130
MYA        71
BRVO       63
ODP        50
CRVO       44
CNV        48
RS         47
ODE        46
LS         37
CSR        29
HTR        28
ASR        26
CRS        24
OTHER     209
dtype: int64

In [3]:
# transform = A.Compose([
#     A.Rotate(limit=45, p=0.5),  # 旋轉，概率為0.5
#     A.HorizontalFlip(p=0.5),    # 水平翻轉，概率為0.5
# ])

# transform = A.Compose([
#     A.OneOf([
#         A.Rotate(limit=45, p=1.0),  # 旋转，当被选择时应用的概率为1.0
#         A.HorizontalFlip(p=1.0),    # 水平翻转，当被选择时应用的概率为1.0
#         A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=1.0),  # 随机亮度对比度调整
#         A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=1.0)  # 色调饱和度调整
#     ], p=1)
# ])

In [7]:
# # images_path = Path("../data/fundus/MuReD/images/images") # training set images path
# # da_images_path = Path("../data/fundus/MuReD/images/ros") # augmented images path

# class_add_count = 100
# ID_name = 1
# while True:
#     class_counts = labels_df.iloc[:, 1:].sum(axis=0)
#     sorted_class_counts = class_counts.sort_values()
#     if all(sorted_class_counts >= class_add_count):
#         break 
    
#     for class_name, count in sorted_class_counts.items():
#         # print(class_name)
#         adds = 0
#         images_to_augment = labels_df[labels_df[class_name] == 1]
#         while count + adds < class_add_count:
#             img_rows = images_to_augment.sample(n=min(class_add_count-count-adds, count))
#             # print(img_rows)
#             for index, img_row in img_rows.iterrows():
#                 # img_name = img_row['ID']
#                 # if img_name.startswith('DA'):
#                 #     continue
                
#                 # image_path = images_path / f"{img_name}.png" if os.path.exists(images_path / f"{img_name}.png") else images_path / f"{img_name}.tif"
#                 # image = cv2.imread(str(image_path))
#                 # if image is not None:
#                 #     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    
#                     # transformed = transform(image=image)
#                     # transformed_image = transformed["image"]

#                     # new_img_name = f"DA_{ID_name}.png"
#                     # ID_name += 1
#                     # new_img_path = da_images_path / new_img_name
#                     # cv2.imwrite(str(new_img_path), cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
                    
#                 new_row = img_row.copy()
#                 adds += 1
#                 # new_row['ID'] = new_img_name.replace('.png', '')
#                 labels_df = labels_df._append(new_row, ignore_index=True)

#         break
#     # print(class_counts)
#     # print("xxxxxxxxxxxxxxxxxxxxxxxx")
            
# class_counts

CRS
ASR
CSR
LS
HTR
ODE
RS
CRVO
ODP
BRVO
CNV
MYA


DR        474
NORMAL    395
MH        147
ODC       248
TSLN      135
ARMD      174
DN        156
MYA       100
BRVO      100
ODP       100
CRVO      100
CNV       100
RS        100
ODE       103
LS        113
CSR       103
HTR       107
ASR       156
CRS       101
OTHER     263
dtype: int64

In [8]:
# labels_df.to_csv('../data/fundus/MuReD/ros100_train_data.csv', index=False)

# Final Data augmentation

In [12]:
import pandas as pd
import numpy as np
from sklearn.utils import resample

# df = pd.read_csv('../data/fundus/ITRI/train_itri.csv')
df = pd.read_csv('../data/fundus/MuReD/train_data.csv')
label_counts = df.iloc[:, 1:].sum(axis=0)

# threshold = label_counts.max() * 0.2
# print(f"---------threshold : {threshold}-----------")

# minority_labels = label_counts[label_counts < threshold].index.tolist()

def augment_data(df, label_counts):
    threshold = label_counts.max() * 0.2
    print(f"---------threshold : {threshold}-----------")
    minority_labels = label_counts[label_counts < threshold].index.tolist()
    needs_augmentation = {label: int(threshold - count) for label, count in label_counts.items() if label in minority_labels and count < threshold}
        
    while any(needs > 0 for needs in needs_augmentation.values()):
        label, needs = max(needs_augmentation.items(), key=lambda item: item[1])
        if needs > 0:
            minority_samples = df[df[label] == 1].copy()
            samples_to_add = resample(minority_samples, n_samples=needs, replace=True)

            df = pd.concat([df, samples_to_add], ignore_index=True)
            label_counts = df.iloc[:, 1:].sum(axis=0)
            needs_augmentation = {label: int(threshold - count) for label, count in label_counts.items() if label in minority_labels and count < threshold}
    return df

df_augmented = augment_data(df, label_counts)

df_augmented.iloc[:, 1:].sum(axis=0)

---------threshold : 8041.0-----------


MHL        9199
ERM        8041
VH         9737
AH         8509
RAO       10364
RVO        8148
ODC       15144
ON         8116
AMD       13470
RT         8149
MYA       27661
CSR        8336
TV        11081
RP        11131
normal     8041
dtype: int64

In [13]:
df_augmented.to_csv('../data/fundus/MuReD/ros02_train_data.csv', index=False)